In [18]:
import convergence as cr
%load_ext autoreload
%autoreload 2
import pandas as pd 
import numpy as np  
import random 
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.stem import WordNetLemmatizer
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Using GloVe

I used word vectors from a pre-trained set of data consisting of <a href="https://catalog.ldc.upenn.edu/LDC2011T07">Gigawords</a> and <a href="https://dumps.wikimedia.org/backup-index.html">2014 Wikimedia dumps</a>.



In [4]:
# import numpy as np
# def load_glove_model(file):
#     print("Loading...")
#     f = open(file,'r')
#     model = {}
#     for line in f:
#         whole_line = line.split()
#         words = whole_line[0]
#         embedding = np.array([float(val) for val in whole_line[1:]])
#         model[words] = embedding
#     print("Done.",len(model)," words loaded!")
#     return model

In [5]:
# model = load_glove_model('glove.6B.300d.txt')

Loading...
Done. 400000  words loaded!


In [6]:
# model['the'].shape

(300,)

In [7]:
# for key, value in model.items():
#     model[key] = value.reshape(1,-1)
# model['the'].shape

(1, 300)

In [9]:
# import pickle 
# # pickle the model
# filename = 'pre_trained_model.pkl'
# pickle.dump(model, open(filename, 'wb'))

In [11]:
model['the'].shape

(1, 300)

Now the GloVe model is pickled and loaded. 

# Common English Words

Create a list of 10,000 common english words from https://github.com/first20hours/google-10000-english. 

In [12]:
# with open('google-10000-english-usa.txt') as f:
#     cwordlist = []
#     for line in f:
#         cwordlist.append(line.strip())
# len(cwordlist)

10000

In [16]:
# #look at the word stems 
# wl = WordNetLemmatizer()

In [17]:
# lemmas = []
# for word in cwordlist: 
#     lemmas.append(wl.lemmatize(word))
# lemmas[9030:]

['grateful',
 'emerald',
 'gradually',
 'laughing',
 'grows',
 'cliff',
 'desirable',
 'tract',
 'ul',
 'ballet',
 'ol',
 'journalist',
 'abraham',
 'j',
 'bumper',
 'afterwards',
 'webpage',
 'religion',
 'garlic',
 'hostel',
 'shine',
 'senegal',
 'explosion',
 'pn',
 'banned',
 'wendy',
 'brief',
 'signature',
 'diffs',
 'cove',
 'mumbai',
 'ozone',
 'discipline',
 'casa',
 'mu',
 'daughter',
 'conversation',
 'radio',
 'tariff',
 'nvidia',
 'opponent',
 'pasta',
 'simplified',
 'muscle',
 'serum',
 'wrapped',
 'swift',
 'motherboard',
 'runtime',
 'inbox',
 'focal',
 'bibliographic',
 'vagina',
 'eden',
 'distant',
 'incl',
 'champagne',
 'ala',
 'decimal',
 'hq',
 'deviation',
 'superintendent',
 'propecia',
 'dip',
 'nbc',
 'samba',
 'hostel',
 'housewife',
 'employ',
 'mongolia',
 'penguin',
 'magical',
 'influence',
 'inspection',
 'irrigation',
 'miracle',
 'manually',
 'reprint',
 'reid',
 'wt',
 'hydraulic',
 'centered',
 'robertson',
 'flex',
 'yearly',
 'penetration',
 'wo

In [18]:
# #find unique lemmas 
# len(lemmas)
# unique_lemmas = set(lemmas)
# len(lemmas)

10000

In [19]:
# #make sure all lemmas are in the vocabulary
# valid_lemmas = []
# invalid_lemmas = [] 

# for word in unique_lemmas:
#     if word in model.keys():
#         valid_lemmas.append(word)
#     else:
#         invalid_lemmas.append(word)

# invalid_lemmas 
# #mostly little-used porn words so i'm fine with them being excluded 

['pichunter',
 'xnxx',
 'blowjob',
 'worldsex',
 'twinks',
 'viewpicture',
 'qty',
 'bufing',
 'mailto',
 'bangbus',
 'verzeichnis',
 'uniprotkb',
 'beastiality',
 'sublimedirectory',
 'handjobs',
 'diffs',
 'smilies',
 'gratuit',
 'milfhunter',
 'trembl',
 'milfs',
 'findarticles',
 'sbjct',
 'voyeurweb',
 'handjob',
 'shemale',
 'knowledgestorm',
 'jelsoft',
 'cooky',
 'trackback',
 'sexcam',
 'msgid',
 'shemales',
 'expansys',
 'powerseller',
 'voyuer',
 'transexuales',
 'basename',
 'endif',
 'dealtime',
 'vsnet',
 'listprice',
 'ampland',
 'msie',
 'livecam',
 'upskirts',
 'titten',
 'upskirt',
 'feof',
 'livesex',
 'postposted',
 'prostores',
 'fioricet',
 'href',
 'holdem',
 'msgstr',
 'trackbacks',
 'telecharger',
 'ultram',
 'acdbentity',
 'techrepublic',
 'adipex',
 'cumshot',
 'changelog',
 'starsmerchant',
 'thumbzilla',
 'thehun',
 'beastality',
 'cumshots',
 'permalink']

In [20]:
# #find small lemmas to make sure they are valid words 
# lemmas_df = pd.DataFrame(valid_lemmas)

In [21]:
# lemmas_df.columns = ['lemma']
# lemmas_df['lemma'].nunique()

8350

In [22]:
# short_lemmas = lemmas_df[lemmas_df['lemma'].apply(lambda x: len(x) < 3)]
# len(short_lemmas)
# short_lemmas.head(15)

,lemma
35,de
44,mn
55,ic
64,dj
86,wt
106,nu
123,pi
147,te
153,us
191,bm


I don't want my bot to start with any of these unknown, smaller words, and it likely won't use them very much. Rather than limiting the bot's response, since these are supposedly the top 8000 most commonly used words, I am going to limit my bot's initial "random" response to be within the first 2000 common words to avoid weird starts to the game. 

In [23]:
# #get vectors for lemmas 
# common_word_vectors = {}
# unvectorized_common_words = []
# for word in valid_lemmas:
#     if word in model.keys():
#         common_word_vectors[word] = model[word]
#     else:
#         unvectorized_common_words.append(word)

In [24]:
# #reshape the dictionary arrays 
# common_word_vectors['truth'].shape

(1, 300)

In [25]:
# for key, value in common_word_vectors.items():
#     common_word_vectors[key] = value.reshape(1,-1)

# common_word_vectors['truth'].shape

(1, 300)

In [26]:
# #pickle the common word vectors 
# file_name = 'common_word_vectors.pkl'
# pickle.dump(common_word_vectors, open(file_name, 'wb'))

In [27]:
common_word_vectors = pickle.load(open('common_word_vectors.pkl', 'rb'))
common_word_vectors['the'].shape

(1, 300)

# Define the Functions

In [22]:
def converge(user_input=str, bot_input=str, exclude=None):
    """Return the "average word" of the input words."""
    if exclude is None:
        exclude = set()
    exclude.add(user_input)
    exclude.add(bot_input)  

    mean_vector = ((model[user_input] + model[bot_input])/2)
    cos_sim_dict = {}
    response_options_dict = {key: common_word_vectors[key] for key in common_word_vectors 
                             if key not in exclude}
    for word, vector in response_options_dict.items():
        cos_sim_dict[float(cosine_similarity(mean_vector, vector))] = word
    max_cos_sim = max(cos_sim_dict.keys())
    bot_response = cos_sim_dict[max_cos_sim]
    return bot_response

In [23]:
def play_round(user_input, user_history=None, bot_history=None):
    """Play a round of Convergence."""
    if bot_history is None:
        user_history = []
        bot_history = []
        bot_response = random.choice(valid_lemmas[:2000])
    else:
        bot_response = converge(user_history[-1], bot_history[-1], 
                                exclude=set(user_history + bot_history))
    user_history.append(user_input)
    bot_history.append(bot_response)
    return {
        'user_history': user_history,
        'bot_history': bot_history,
        'bot_response': bot_response,
    }

In [24]:
def play_convergence(round_results=None):
    """Play a game of Convergence."""
    print("Let's play Convergence. Type any word!") 
    while True: 
        user_response = input()
        if round_results is None:
            bot_history = None 
            user_history = None
            round_results = play_round(user_response, user_history, bot_history)    
            #this will practically never happen, but just in case: 
            if user_response != round_results['bot_response']:
                print(f"You said {user_response}, but I said {round_results['bot_response']}.")
                print(f"What's the convergence of {user_response} and {round_results['bot_response']}?")   

            else:
                print(f"CONVERGENCE!!!! WE BOTH said {user_response}!!! WE GOT CONVERGENCE!!!!")
                break              
        else: 
            bot_history = round_results['bot_history']
            user_history = round_results['user_history']
            
            #ensure user_response is a real word: 
            if user_response in bot_history or user_response in user_history: 
                print(f"Looks like one of us has already said {user_response}. Choose a different word.")
            elif user_response in model.keys():  

                round_results = play_round(user_response, user_history, bot_history)    
                if user_response != round_results['bot_response']:
                    print(f"You said {user_response}, but I said {round_results['bot_response']}.")
                    print(f"What's the convergence of {user_response} and {round_results['bot_response']}?")   
                    if len(round_results['bot_history'])>1:
                          print(f"Remember, don't repeat what you said: {user_history} or what I said: {bot_history}")
                else:
                    print(f"CONVERGENCE!!!! WE BOTH said {user_response}!!! WE GOT CONVERGENCE!!!!")
                    break              
            else: 
                print (f"Is {user_response} an English word? I don't know it. Choose a different word.")

In [25]:
play_convergence()

Let's play Convergence. Type any word!
hello
You said hello, but I said arrangement.
What's the convergence of hello and arrangement?
flower
You said flower, but I said hey.
What's the convergence of flower and hey?
Remember, don't repeat what you said: ['hello', 'flower'] or what I said: ['arrangement', 'hey']
hay
You said hay, but I said lovely.
What's the convergence of hay and lovely?
Remember, don't repeat what you said: ['hello', 'flower', 'hay'] or what I said: ['arrangement', 'hey', 'lovely']
farm
You said farm, but I said beautiful.
What's the convergence of farm and beautiful?
Remember, don't repeat what you said: ['hello', 'flower', 'hay', 'farm'] or what I said: ['arrangement', 'hey', 'lovely', 'beautiful']
erger
Is erger an English word? I don't know it. Choose a different word.
farm
Looks like one of us has already said farm. Choose a different word.
beatuiful
Is beatuiful an English word? I don't know it. Choose a different word.
beautiful
Looks like one of us has alread